In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

from myBuilding import runMyBEM
from runMyBuildingMC import getConstructions, getWeatherData, setConstructionType
from model.WallSimulation import convectionDOE2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
chosenZone = 9
chosenMonth = 8
data, climate_zones = getWeatherData()

## Energy Plus

In [ ]:
dirEP = "/Users/nicholasbachand/codes/plan2EPlus/cases/graphbem_cali"
pathEP = f"{dirEP}/{climate_zones[chosenZone]['WeatherFile']}/surface_data.csv"
print(f"Querying EnergyPlus results for zone {chosenZone} and month {chosenMonth} at {pathEP}")
outputsEP = pd.read_csv(pathEP, index_col="datetimes", parse_dates=True)
outputsEP.index = outputsEP.index.map(lambda dt: dt.replace(year=2008))
print(f"Energy plus outputs: {outputsEP.columns.tolist()}")

for key in outputsEP.keys():
    if "temperature" in key.lower():
        outputsEP[key] += 273.15

outputsEP

In [ ]:
plt.figure(figsize=(10, 6))
sns.lineplot(
    data=outputsEP,
    x='datetimes',
    y='Surface Inside Face Adjacent Air Temperature',
    hue = "zone",
    estimator=None
)

sns.lineplot(
    data=outputsEP,
    x='datetimes',
    y='Surface Outside Face Outdoor Air Drybulb Temperature',
    label=f'outside air temp',
    linestyle=':',
    color='black',
    estimator=None
)
plt.tight_layout()

In [ ]:
quants = [
    "Convection Heat Transfer Coefficient", 
    "Face Temperature", 
    "Radiation Rate per Area"
]
is_exterior = True
for quant in quants:
    keys = []
    for key in outputsEP.keys():
        if quant in key:
            keys.append(key)

    n_keys = len(keys)
    fig, ax = plt.subplots(n_keys, figsize=(10, 3*n_keys), sharex=True)
    ax = ax.flatten() if n_keys > 1 else [ax]
    for i, key in enumerate(keys):
        sns.lineplot(
            data=outputsEP[outputsEP['is_exterior'] == is_exterior],
            x='datetimes',
            y=key,
            estimator=None,
            hue="direction",
            style="zone",
            ax=ax[i]
        )
        ax[i].set_title(key)
        ax[i].set_ylabel("")
    plt.tight_layout()

## Graph BEM

In [ ]:
material_type = "Heavy"
materials = getConstructions("My", constructionFile = "energyPlus/My_Constructions.csv")
chosenMaterial = setConstructionType(materials, material_type)
chosenMaterial

In [ ]:
dt = 30
dataSampled = data[(data["ClimateZone"] == chosenZone) & (data.index.month == chosenMonth)]
dataSampled = dataSampled.iloc[1:]
dataSampled = dataSampled.infer_objects(copy=False)
dataSampled = dataSampled.resample(f"{dt}s").interpolate()

In [ ]:
dataSampled["Dry Bulb Temperature"][1:]

In [ ]:
plt.figure(figsize=(10, 6))
sns.lineplot(
    data=outputsEP,
    x='datetimes',
    y='Surface Outside Face Outdoor Air Drybulb Temperature',
    label='EP Outside Air Temp',
    estimator=None
)

In [ ]:
floorTempAdjustment = np.mean([-3.5, -5])
hInterior = np.mean([1, 3])
alphaRoof = np.mean([0.6, 0.9])
windSpeed = np.mean([0, 6])
windSpeed = np.mean([0, 6])
wallRoughness = np.mean([1.11, 2.17])
hExterior = convectionDOE2(np.mean([1, 3]), windSpeed, wallRoughness) #using DOE-2 to calculate this

In [ ]:
outputsGraphBEM = runMyBEM(
    dataSampled,
    materials,
    floorTempAdjustment,
    hInterior,
    hExterior,
    alphaRoof,
    verbose = True,
    makePlots = True)

In [ ]:
outputsGraphBEM["ceilingMinusFloor"]